In [3]:
import shap
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
%matplotlib notebook

from envs.trading_env_integrated import TradingEnv
import DQNTradingAgent.dqn_agent as dqn_agent

device = torch.device('cpu')
dqn_agent.set_device(device)

from tqdm import tqdm, tnrange
import pickle

/home/jeffrey/Binanace_trading_simulation/agent/DQN/envs/trading_env_integrated.py:4: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'nbAgg' by the following code:
  File "/home/jeffrey/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/jeffrey/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jeffrey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/jeffrey/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/jeffrey/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.sta

In [4]:
n_action_intervals = 5

sample_len   = 480
obs_data_len = 192
step_len     = 1
fee          = 0.0
sell_at_end  = False

df = pd.read_hdf('dataset/binance_data_test.h5', 'STW')
df.fillna(method='ffill', inplace=True)

In [7]:
hyperparam_path = f"./saves/transfer/rsi/1/hyperparams.pth"
statedict_path  = f"./saves/transfer/rsi/1/TradingGym_Rainbow_400.pth"

In [8]:
 class args():
        def __init__(self, env_name):
            self.environment = env_name

args = args('rsi')

env = TradingEnv(custom_args=args, env_id='custom_trading_env', obs_data_len=obs_data_len, step_len=step_len, sample_len=sample_len,
                         df=df, fee=fee, initial_budget=1, n_action_intervals=n_action_intervals, deal_col_name='c',
                         sell_at_end=sell_at_end,
                         feature_names=['o', 'h','l','c','v',
                                        'num_trades', 'taker_base_vol'])

agent = dqn_agent.Agent(action_size=2 * n_action_intervals + 1,
                            obs_len=obs_data_len,
                            num_features=env.reset().shape[-1],
                            **torch.load(hyperparam_path))

agent.qnetwork_local.load_state_dict(torch.load(statedict_path, map_location=device))
agent.qnetwork_local.eval()
for param in agent.qnetwork_local.parameters():
    param.requires_grad_(False)

[2019-09-06 17:38:38,410] Making new env: custom_trading_env


In [10]:
next_state = env.reset()
state_batch=[]
state_batch.append(next_state)
for i in tnrange(1000):
    state = next_state
    state_batch.append(state)
    action = int(agent.act(state, eps=0.))
    next_state, _, done, _, fee_rate = env.step(action)
    if done:
        break

In [59]:
model=agent.qnetwork_local
state=torch.tensor(state_batch[:10])

In [82]:
test=torch.tensor(state_batch[10:12])
test.shape

torch.Size([2, 192, 8])

In [83]:
e = shap.DeepExplainer(model,state.float())

In [84]:
shap_values = e.shap_values(test.float())

RuntimeError: grad can be implicitly created only for scalar outputs